# Data Preprocessing - Home Credit Default Risk (Selected Features)

## Goal
This notebook shows how to clean and prepare data for machine learning using only the most important features.

## Steps:
1. Load data and select important features
2. Handle missing values
3. Encode categorical variables
4. Scale numerical features
5. Save processed data

---
